In [ ]:
#Loading the necessary packages!
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import optuna
import xgboost as xgb
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVC
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, accuracy_score, median_absolute_error
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import KFold
from scipy import stats
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import catboost as cb
from scipy.optimize import minimize

In [ ]:
#Loasding the data!!
train = pd.read_csv("/kaggle/input/bagpack11/train.csv")
test = pd.read_csv("/kaggle/input/bagpack11/test.csv")
extra_train = pd.read_csv("/kaggle/input/bagpack11/training_extra.csv")


In [ ]:
train.head(10)

In [ ]:
test.head(10)

In [ ]:
extra_train.head(10)

In [ ]:
#checking the number of rows and column present in the data

num_train_rows, num_train_columns = train.shape

num_test_rows, num_test_columns = test.shape

num_extra_rows, num_extra_columns = extra_train.shape

print("Training Data:")
print(f"Number of Rows: {num_train_rows}")
print(f"Number of Columns: {num_train_columns}\n")

print("Test Data:")
print(f"Number of Rows: {num_test_rows}")
print(f"Number of Columns: {num_test_columns}\n")

print("Extra Data:")
print(f"Number of Rows: {num_extra_rows}")
print(f"Number of Columns: {num_extra_columns}")

This below code creates a summary table that displays information about missing values, unique values, and data types for features in multiple datasets (train, test, extra_train)

In [ ]:
# Creating a table for missing values, unique values and data types of the features

missing_values_train = pd.DataFrame({'Feature': train.columns,
                              '[TRAIN] No. of Missing Values': train.isnull().sum().values,
                              '[TRAIN] % of Missing Values': ((train.isnull().sum().values)/len(train)*100)})

missing_values_test = pd.DataFrame({'Feature': test.columns,
                             '[TEST] No.of Missing Values': test.isnull().sum().values,
                             '[TEST] % of Missing Values': ((test.isnull().sum().values)/len(test)*100)})

missing_values_original = pd.DataFrame({'Feature': extra_train.columns,
                             '[ORIGINAL] No.of Missing Values': extra_train.isnull().sum().values,
                             '[ORIGINAL] % of Missing Values': ((extra_train.isnull().sum().values)/len(extra_train)*100)})

unique_values = pd.DataFrame({'Feature': train.columns,
                              'No. of Unique Values[FROM TRAIN]': train.nunique().values})
feature_types = pd.DataFrame({'Feature': train.columns,
                              'DataType': train.dtypes})

merged_df = pd.merge(missing_values_train, missing_values_test, on='Feature', how='left')
merged_df = pd.merge(merged_df, missing_values_original, on='Feature', how='left')
merged_df = pd.merge(merged_df, unique_values, on='Feature', how='left')
merged_df = pd.merge(merged_df, feature_types, on='Feature', how='left')

merged_df

In [ ]:
#Count the duplicates in the train test and extra train 
train_duplicates = train.duplicated().sum()
test_duplicates = test.duplicated().sum()
extra_duplicates = extra_train.duplicated().sum()

print(f"Number of duplicate rows in train data: {train_duplicates}")
print(f"Number of duplicate rows in test data: {test_duplicates}")
print(f"Number of duplicate rows in extra train data: {extra_duplicates}")


In [ ]:
train.describe().T

EDA

In [ ]:
train = pd.concat([train, extra_train], ignore_index=True)


In [ ]:
numerical_variables = ["Weight Capacity (kg)"]
target_variable = "Price"
categorical_variables = ['Brand', 'Material', 'Size', 'Compartments', 'Laptop Compartment','Waterproof', 'Style', 'Color']

Data Imputation (Handling missing values)

Understanding Each Feature and Missing Data Handling
1️⃣ Brand (~3.24% missing in train, ~3.11% in test)
Unique Values: Jansport, Under Armour, Nike, Adidas, Puma, NaN
Imputation Strategy: Since it's categorical with only 5 unique brands, we can impute missing values with the mode (most frequent brand).
2️⃣ Material (~2.78% missing in train, ~2.81% in test)
Unique Values: Leather, Canvas, Nylon, Polyester, NaN
Imputation Strategy: Mode imputation works best here since materials are limited categories.
3️⃣ Size (~2.20% missing in train, ~2.19% in test)
Unique Values: Medium, Small, Large, NaN
Imputation Strategy: Since it's a well-defined categorical variable, we use mode imputation.
4️⃣ Compartments (✅ No missing values)
Unique Values: 10 unique numerical values
Imputation Strategy: ✅ No action needed.
5️⃣ Laptop Compartment (~2.48% missing in train, ~2.48% in test)
Unique Values: Yes, No, NaN
Imputation Strategy: Since it's a binary categorical variable, we use mode imputation.
6️⃣ Waterproof (~2.35% missing in train, ~2.41% in test)
Unique Values: Yes, No, NaN
Imputation Strategy: Mode imputation is best.
7️⃣ Style (~2.66% missing in train, ~2.58% in test)
Unique Values: Tote, Messenger, Backpack, NaN
Imputation Strategy: Mode imputation.
8️⃣ Color (~3.32% missing in train, ~3.39% in test)
Unique Values: Black, Green, Red, Blue, Gray, Pink, NaN
Imputation Strategy: Mode imputation.
9️⃣ Weight Capacity (kg) (~0.05% missing in train, ~0.04% in test)
Unique Values: Numeric
Imputation Strategy: Since it is a continuous numerical variable, we use median imputation to prevent extreme values from affecting the distribution.

In [ ]:
# Define imputation strategies
categorical_features = ["Brand", "Material", "Size", "Laptop Compartment", "Waterproof", "Style", "Color"]
numerical_features = ["Weight Capacity (kg)"]

# Fill categorical missing values with mode (most frequent value)
for col in categorical_features:
    train[col].fillna(train[col].mode()[0], inplace=True)
    test[col].fillna(test[col].mode()[0], inplace=True)

# Fill numerical missing values with median
for col in numerical_features:
    train[col].fillna(train[col].median(), inplace=True)
    test[col].fillna(test[col].median(), inplace=True)

In [ ]:
train.isnull().sum()

In [ ]:
from sklearn.model_selection import train_test_split

# Split the dataset into train and test sets
X = train.drop(columns='Price')  # Assuming 'Price' is the target
y = train['Price']




In [ ]:
# Define categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object']).columns
numerical_cols = X.select_dtypes(exclude=['object']).columns
print(categorical_cols)
print(numerical_cols)


In [ ]:
# Encode categorical variables using Label Encoding
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col])
    X_test[col] = le.transform(X_test[col])  # Important to use the same encoding for both train and test sets
    label_encoders[col] = le


In [ ]:
# from sklearn.pipeline import Pipeline
# from sklearn.model_selection import KFold
# from sklearn.metrics import mean_squared_error
# import numpy as np
# import pandas as pd
# from sklearn.linear_model import LinearRegression  # Import Linear Regression

# # Ensure preprocessor is defined (Replace with actual preprocessing steps)
# # Example: preprocessor = SomePreprocessingStep()
# assert 'preprocessor' in globals(), "Define 'preprocessor' before using it in the pipeline."

# # Initialize KFold
# n_folds = 5
# kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

# # DataFrame to store out-of-fold predictions
# oof_df = pd.DataFrame(columns=['ID', 'Actual', 'OOF_Pred_LR', 'Fold'])

# # Convert X_train and y_train to DataFrames if they aren't already
# X_train = pd.DataFrame(X_train)
# y_train = pd.Series(y_train)

# # Initialize array to store out-of-fold predictions
# oof_preds = np.zeros(len(X_train))

# # K-Fold Cross-Validation Loop
# for fold, (train_idx, val_idx) in enumerate(kf.split(X_train, y_train), start=1):
#     print(f"\n🔄 Training Fold {fold}/{n_folds}...")

#     # Split data into training and validation sets
#     X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
#     y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

#     # Create pipeline with preprocessor and Linear Regression model
#     pipeline = Pipeline(steps=[
#         ('preprocessor', preprocessor),  # Replace with actual preprocessor
#         ('model', LinearRegression())  # Replace with Linear Regression
#     ])

#     # Train the model
#     pipeline.fit(X_tr, y_tr)

#     # Predict on validation set (OOF predictions)
#     y_val_pred = pipeline.predict(X_val)
#     oof_preds[val_idx] = y_val_pred  # Store OOF predictions

#     # Compute fold RMSE
#     fold_rmse = mean_squared_error(y_val, y_val_pred, squared=False)
#     print(f"✅ Fold {fold} RMSE: {fold_rmse:.4f}")

#     # Store fold results in DataFrame
#     fold_df = pd.DataFrame({
#         'ID': X_train.index[val_idx],  # Assuming index represents unique IDs
#         'Actual': y_val.values,
#         'OOF_Pred_LR': y_val_pred,
#         'Fold': fold
#     })

#     oof_df = pd.concat([oof_df, fold_df], ignore_index=True)

# # Compute overall OOF RMSE
# oof_rmse = mean_squared_error(y_train, oof_preds, squared=False)
# print(f"\n🏆 Overall OOF RMSE: {oof_rmse:.4f}")

# # Save OOF predictions to CSV
# oof_df.to_csv('oof_predictions_lr.csv', index=False)
# print("📂 OOF predictions saved to 'oof_predictions_lr.csv'.")

# # Display first few rows of OOF predictions
# oof_df.reset_index(drop=True, inplace=True)
# print(oof_df.head())


In [ ]:
# # Make predictions on the test set
# predictions = pipeline.predict(test)

# # Create the submission DataFrame
# submission = pd.DataFrame({
#     'id': test['id'],         # Ensure 'id' exists in the test set
#     'Price': predictions      # Use predictions on the test set
# })

# # Save the DataFrame to a CSV file
# submission.to_csv('submission200.csv', index=False)

In [ ]:
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.impute import SimpleImputer
# from sklearn.pipeline import Pipeline
# from sklearn.model_selection import KFold, GridSearchCV
# from sklearn.metrics import mean_squared_error
# from sklearn.linear_model import Ridge
# import numpy as np
# import pandas as pd

# # Ensure preprocessor is defined (Replace with actual preprocessing steps)
# assert 'preprocessor' in globals(), "Define 'preprocessor' before using it in the pipeline."

# # Define columns
# numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns
# categorical_features = X_train.select_dtypes(include=['object']).columns

# # Create numeric transformer
# numeric_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='mean'))  # Impute missing values with mean for numeric features
# ])

# # Create categorical transformer
# categorical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='most_frequent')),  # Impute missing values with most frequent for categorical features
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))  # One-hot encoding for categorical features
# ])

# # Combine both transformations using ColumnTransformer
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numeric_transformer, numeric_features),
#         ('cat', categorical_transformer, categorical_features)
#     ])

# # Initialize KFold
# n_folds = 5
# kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

# # DataFrame to store out-of-fold predictions
# oof_df = pd.DataFrame(columns=['ID', 'Actual', 'OOF_Pred_Ridge', 'Fold'])

# # Convert X_train and y_train to DataFrames if they aren't already
# X_train = pd.DataFrame(X_train)
# y_train = pd.Series(y_train)

# # Initialize array to store out-of-fold predictions
# oof_preds = np.zeros(len(X_train))

# # Ridge hyperparameter grid for tuning alpha
# param_grid = {
#     'model__alpha': [0.01, 0.1, 1, 10, 100]  # Regularization strength (alpha)
# }

# # K-Fold Cross-Validation Loop
# for fold, (train_idx, val_idx) in enumerate(kf.split(X_train, y_train), start=1):
#     print(f"\n🔄 Training Fold {fold}/{n_folds}...")

#     # Split data into training and validation sets
#     X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
#     y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

#     # Create pipeline with imputation, preprocessor, and Ridge regression model
#     pipeline = Pipeline(steps=[
#         ('preprocessor', preprocessor),  # Apply transformations to numeric and categorical features
#         ('model', Ridge())  # Use Ridge regression instead of Linear Regression
#     ])

#     # Set up GridSearchCV for hyperparameter tuning
#     grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')

#     # Train the model using grid search
#     grid_search.fit(X_tr, y_tr)

#     # Best model after tuning
#     best_model = grid_search.best_estimator_

#     # Predict on validation set (OOF predictions) using the best model
#     y_val_pred = best_model.predict(X_val)
#     oof_preds[val_idx] = y_val_pred  # Store OOF predictions

#     # Compute fold RMSE
#     fold_rmse = mean_squared_error(y_val, y_val_pred, squared=False)
#     print(f"✅ Fold {fold} RMSE: {fold_rmse:.4f}")

#     # Store fold results in DataFrame
#     fold_df = pd.DataFrame({
#         'ID': X_train.index[val_idx],  # Assuming index represents unique IDs
#         'Actual': y_val.values,
#         'OOF_Pred_Ridge': y_val_pred,
#         'Fold': fold
#     })

#     oof_df = pd.concat([oof_df, fold_df], ignore_index=True)

# # Compute overall OOF RMSE
# oof_rmse = mean_squared_error(y_train, oof_preds, squared=False)
# print(f"\n🏆 Overall OOF RMSE: {oof_rmse:.4f}")

# # Save OOF predictions to CSV
# oof_df.to_csv('oof_predictions_ridge.csv', index=False)
# print("📂 OOF predictions saved to 'oof_predictions_ridge.csv'.")

# # Display first few rows of OOF predictions
# oof_df.reset_index(drop=True, inplace=True)
# print(oof_df.head())


In [ ]:
# # After training and obtaining out-of-fold predictions (as you're already doing)
# # Make predictions on the test set (Replace 'test' with your actual test DataFrame)
# test = pd.DataFrame(test)  # Ensure test is a DataFrame
# assert 'id' in test.columns, "The test DataFrame must contain an 'id' column."

# # Make predictions on the test set
# test_predictions = best_model.predict(test)  # Make predictions using the best model after GridSearchCV

# # Create the submission DataFrame
# submission = pd.DataFrame({
#     'id': test['id'],         # Ensure 'id' exists in the test set
#     'Price': test_predictions  # Use predictions on the test set
# })

# # Save the DataFrame to a CSV file
# submission.to_csv('submissiono0101.csv', index=False)

# # Save the OOF predictions DataFrame to a CSV file
# oof_df.to_csv('oof_predictions_ridge.csv', index=False)
# print("📂 OOF predictions saved to 'oof_predictions_ridge.csv'.")
